# Main solution notebook - IBM Quantum Awards: Open Science Prize 2021 - QTime solution

__________

### QTime: *André Juan, Anton Simen, Askery Canabarro, Rafael Chaves, Rodrigo Pereira*

__________

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

MARKDOWN

__________

In [ ]:
from troter_utils import *

## Quantum Devices


In [ ]:
provider = IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider.get_backend('ibmq_jakarta')

sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))

____________

## Classical optimization of variational quantum circuit

Each Trotter step is parametrized by a free "time" parameter.

The best results were obtained with the folowing experiment:

- First order trotterization;
<br><br>
- $t_{\text{min}} =-\pi$;
    - That is, $t_{\text{step}} \in (-\pi, \pi)$;
<br><br>
- n_steps $\in \{4, 5, 6, 7, 8\}$.

So, this will be the parameters set for the optimization below.

Of course, though, any other combination of input parameters may be tested!

(Notice that, due to changes in error rates between callibrations, the best results may change. This is a natural part of working with NISQ hardware. Such variations were observed in the period of the solution development, and may be observed again, whenever the full pipeline runs again).

In [ ]:
results = {"order" : [],
           "n_steps" : [],
           "t_min" : [],
           "state_tomo_fids" : [],
           "fid_pi" : [],
           "best_params" : []}

In [ ]:
# parameters of the desired experiment are set here!
# parameters were chosen as discussed above.
# feel free to change the parameters to try out different experiments!
order = 1
uniform_times = False
backend_opt, backend_state_tomo = sim_noisy_jakarta, sim_noisy_jakarta
quadratic_loss = False
steps = range(4, 9)
min_times = [-np.pi]

combs = itertools.product(steps, min_times)

for trotter_steps, params_bounds_min in combs:
    
    print("\n\n")    
    print("#"*80)
    print("="*80)
    print("#"*80)
    print(f"Order: {order}".center(80))
    print(f"# steps: {trotter_steps}".center(80))
    print(f"min time: {params_bounds_min}".center(80))
    print("#"*80)
    print("="*80)
    print("#"*80)
    print("\n\n")
    
    fids, fidelity_pi, best_params = optimize_params_and_run(order, trotter_steps, uniform_times, params_bounds_min,
                                                             backend_opt, backend_state_tomo, quadratic_loss)
    
    results['order'].append(order)
    results['n_steps'].append(trotter_steps)
    results['t_min'].append(params_bounds_min)
    results['state_tomo_fids'].append(fids)
    results['fid_pi'].append(fidelity_pi)
    results['best_params'].append(best_params)

As you might have noticed, the classical optimization above may take some time. However, it is not necessary to perform it everytime that a job is submited for hardware execution: we will save its results to a parquet file (identified with the date, so that it's not overwritten if another optimization is desired later on). With that, one can simply skip to the "Hardware Execution" section, read the existing results file(s), and send jobs to execution, without having to re-run the classical optimization again (although this may be desirable, if one wants to optimize the parameters using he jakarta simulator with the same callibration settings that will be encountered in the actual hardware).

Obs.: files were saved as parquet, because it allows for an easier parse of columns of lists (best_params, eg). You must have `pyarrow` or `fastparquet` installed to work with this kind of file.

### Structuring and saving classical optimization results

In [ ]:
df_results = pd.DataFrame(results).sort_values("fid_pi", ascending=False)
df_results["fid_pi"] = df_results["fid_pi"].astype("float")

# let's save in the same folder (so it won't be mixed with previous results)
file_results = f"results_judge_test_{dt.datetime.today().date()}.parquet"
df_results.to_parquet(file_results, index=False)

df_results

___________

## Hardware execution

After we have the optimized parameters for all combinations above, we'll submit the jobs for hardware execution.

We'll read the file generated above, for completeness.

Depending on the date in which this code is executed by the judges (and the amount of different dates), we may have different results. So, let's show a list of them all:

In [ ]:
results_files = [f for f in listdir("./") if isfile(join("./", f))]

results_judge = [x for x in results_files if "results_judge_test" in x]

results_judge

In [ ]:
# please set the string below with the desired date
# IMPORTANT: it must be in the format "YYYY-MM-DD", 
# as specified in the code which generates the results file!
# (you may use the list above to see all the available dates)
str_date = ""

In [ ]:
# check
if str_date == "":
    raise ValueError("\n\nPlease, set the date string, as instructed above!!\n")

In [ ]:
df_results_read = pd.read_parquet(f'./results/results_opt_first_order_{str_date}.parquet')

df_results_read

_____________

### Sending jobs for execution, and saving pickle file with jobs IDs

In [ ]:
jobs_dict = send_jobs(df_results_read, jakarta, uniform_times=False)

jobs_dict

In [ ]:
jobs_dict_ids = {}

for k, v in jobs_dict.items():
    
    ids = [x._job_id for x in v]
    
    jobs_dict_ids[k] = ids
    
with open(f'dict_jobs_ids_judge_test_{str_date}.pkl', 'wb') as f:
    pickle.dump(jobs_dict_ids, f)
    
jobs_dict_ids

_____________

### Retrieving hardware execution results, and producing final results

In [ ]:
final_results_analysis(f'dict_jobs_ids_judge_test_{str_date}.pkl', jakarta, print_all_details=False, save_here=True)

_____________

# And that's it! 